In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

# Uruchomienie przeglądarki
driver = webdriver.Edge()
URL = "https://www.filmweb.pl/ranking/vod/serial/2024"
driver.get(URL)
time.sleep(3)  # Czekamy na załadowanie strony

# Akceptacja cookies
try:
    accept_button = driver.find_element(By.ID, "didomi-notice-agree-button")
    accept_button.click()
    time.sleep(2)
except:
    print("Brak przycisku cookies.")

# Scrollowanie strony
scroll_amount = 1000
for _ in range(15):  # Scrollujemy kilka razy
    driver.execute_script(f"window.scrollBy(0, {scroll_amount});")
    time.sleep(2)

# Pobranie kodu HTML po załadowaniu
page_source = driver.page_source
driver.quit()  # Zamykamy Selenium, resztę robi BeautifulSoup

# Parsowanie HTML
soup = BeautifulSoup(page_source, "html.parser")

# Wyszukiwanie elementów
serials = []
for item in soup.find_all("div", class_="rankingType"):
    try:
        title = item.find("h2", class_="rankingType__title").text.strip()
        rating = item.find("span", class_="rankingType__rate--value").text.strip()
        
        # Pobieranie linku do okładki
        cover_image_tag = item.find("div", class_="efficientPoster").find("img")
        cover_image_url = cover_image_tag["src"] if cover_image_tag else None
        
        # Pobieranie gatunków
        genres = item.find("div", class_="rankingType__genres")
        genres_list = [g.text.strip() for g in genres.find_all("a", class_="tag")] if genres else []
        genres_str = ", ".join(genres_list)

        serials.append([title, rating, cover_image_url, genres_str])
    except Exception as e:
        print(f"Błąd: {e}")

# Tworzenie DataFrame
df = pd.DataFrame(serials, columns=["Tytuł", "Ocena", "Link do okładki", "Gatunek"])

# ✅ Zapis do pliku CSV
df.to_csv("filmweb_serials.csv", index=False, encoding="utf-8")

# ✅ Wyświetlenie DataFrame w notebooku
df.head()

,Tytuł,Ocena,Link do okładki,Gatunek
0,1. X-Men '97,"8,06",https://fwcdn.pl/fpo/66/29/10016629/8119717_1....,"Animacja, Akcja, Sci-Fi"
1,2. Pingwin,"8,06",https://fwcdn.pl/fpo/02/62/10010262/8143252.7....,Kryminał
2,3. Dan Da Dan,"7,81",https://fwcdn.pl/fpo/89/86/10058986/8139984.7....,"Akcja, Sci-Fi, Anime"
3,4. Królowa łez,"7,78",https://fwcdn.pl/fpo/82/25/10038225/8115080_1....,Komedia rom.
4,5. Szōgun,"7,76",https://fwcdn.pl/fpo/50/08/10025008/8111087_1....,"Przygodowy, Dramat historyczny"
